<a href="https://colab.research.google.com/github/aziesel/Coursera_Capstone/blob/master/ADSC_week3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Start by installing BeautifulSoup (bs4 package) and importing other required modules: requests for retrieving HTML data, and pandas for creating a data frame.

In [3]:
#!conda install -c conda-forge bs4 --yes
import requests
from bs4 import BeautifulSoup
import pandas as pd


Firstly, we use request to retrieve the contents of the specified Wikipedia page. Then we create a BeautifulSoup object ('to_soup') to parse out the table containing information and specifically the row contents ('to_postal' and 'to_postal_table'). 

For each row in to_postal_table, we collect each of the entries on that row (marked with <td> </td> flags in the HTML), remove any newline characters, and append each row as one element into a temporary list.

Next, we convert the temporary list to a DataFrame object, remove an errant empty row, and remove any postal codes for which the Borough or Neighbourhood is 'Not assigned'. Finally we reset the index of the DataFrame to adjust for removed rows.

In [21]:
to_data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
to_soup = BeautifulSoup(to_data, "html5lib")
to_postal = to_soup.find('table', {'class':'wikitable sortable'})
to_postal_table = to_postal.find_all('tr')

temp = []
for row in to_postal_table:
    entry = row.find_all('td')
    row = [row.text.strip() for row in entry]
    temp.append(row)

to_df = pd.DataFrame(temp, columns=["PostalCode", "Borough", "Neighbourhood"])
to_df = to_df.iloc[1:]
to_df = to_df[to_df.Borough != 'Not assigned']
to_df = to_df[to_df.Neighbourhood != 'Not assigned']
to_df.reset_index()
print(to_df)

    PostalCode  ...                                      Neighbourhood
3          M3A  ...                                          Parkwoods
4          M4A  ...                                   Victoria Village
5          M5A  ...                          Regent Park, Harbourfront
6          M6A  ...                   Lawrence Manor, Lawrence Heights
7          M7A  ...        Queen's Park, Ontario Provincial Government
..         ...  ...                                                ...
161        M8X  ...      The Kingsway, Montgomery Road, Old Mill North
166        M4Y  ...                               Church and Wellesley
169        M7Y  ...  Business reply mail Processing Centre, South C...
170        M8Y  ...  Old Mill South, King's Mill Park, Sunnylea, Hu...
179        M8Z  ...  Mimico NW, The Queensway West, South of Bloor,...

[103 rows x 3 columns]


Lastly, the shape of our DataFrame is:

In [ ]:
to_df.shape

(103, 3)